<a href="https://colab.research.google.com/github/SeanSDarcy2001/games/blob/main/dominoRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
#imports
import numpy as np
import random
from itertools import cycle, islice
from collections import deque

In [60]:
#objects

class Player() :

  def __init__(self, Name, intelligent = False) :
    self.name = Name
    self.hand = []
    if intelligent :
      self.intel = True

  def obtainHand(self, sevenDominos) :
    for doms in sevenDominos :
      self.hand.append(doms)

  def displayHand(self) :
    print(self.name, " has ", str(len(self.hand)), " dominos left.")
    for doms in self.hand :
      print("%%%%%%")
      doms.printDom()

  def placeDomino(self, board) :
    a, b = board.getEnds()
    if self.intel :
      print("TOIMPLEMENT")
    else :
      for doms in self.hand
        val1, val2 = doms.getsVals()
        if val1 == a or val1 == b or val2 == a or val2 == b
        #NEED to think this out


class Board() :
  def __init__(self) :
    self.board = deque()

  def addDomino(self, domino, side) :
    if side == "A" :
      self.board.append(domino)
    else :
      self.board.appendleft(domino)

  def getEnds(self) :
    dominoA = self.board[0]
    dominoB = self.board[-1]
    a, c = dominoA.getVals() #care about a from a
    d, b = dominoB.getVals() #care about b from b
    return a, b

class Domino() :

  def __init__(self, a, b) :
    self.val1 = a
    self.val2 = b

  def printDom(self) :
    print(self.val1)
    print("=")
    print(self.val2)

  def getVals(self) :
    return self.val1, self.val2

class DominoSet() :
  def __init__(self) :
    self.box = []
    for i in range(7) :
      for j in range(7) :
        if j >= i :
          self.box.append(Domino(i, j))

  def printBox(self) :
    for doms in self.box :
      print("%%%%%%")
      doms.printDom()

  def giveToPlayer(self) :
    playerDoms = []
    for k in range(7) :
      random_dom = random.choice(self.box)
      self.box.remove(random_dom)
      playerDoms.append(random_dom)
    return playerDoms

class gameSimulation() :

  def __init__(self, playersInGame) :
    self.players = playersInGame
    dominos = DominoSet()

    for player in self.players :
      fromBox = dominos.giveToPlayer() #gets 7 seven random dominos
      for doms in fromBox : #checks whos got the 6\\6
        a, b = doms.getsVals
        if a == 6 and b == 6 :
          self.startingPlayer = player
      player.obtainHand(fromBox) #gives dominos to player

    #reorder player list based on starting player
    self.players = self.reorder_players(self.players.index(self.startingPlayer))

  def reorder_players(self, index):
    if index >= len(self.players):
        return self.players  # Index out of range, return the original list
    # Split the list into two parts
    first_part = self.players[:index]
    second_part = self.players[index:]
    # Concatenate the second part with the first part
    reordered_lst = second_part + first_part
    return reordered_lst

  def runGame(self) :
    running = True
    for player in cycle(self.players)
      if running == False : #terminal condition for once game ends
        break








SyntaxError: ignored

In [40]:
#create domino set, display set and set size
dominos = DominoSet()
print("DOMINO SET")
print("=============")
dominos.printBox()
print("=============")
print("DOMINO SET SIZE")
print("=============")
print(len(dominos.box))
print("=============")

#define players and deal seven random dominos each
players = [Player("Seby", intelligent = True), Player("Mino"), Player("Tita"), Player("Gordo")]
for player in players :
  fromBox = dominos.giveToPlayer() #gets 7 seven random dominos
  player.obtainHand(fromBox) #gives dominos to player

#display domino set after dealing, should be empty
print("POST DEAL DOMINO SET")
print("=============")
dominos.printBox()
print("=============")
print("POST DEAL DOMINO SET SIZE")
print("=============")
print(len(dominos.box))
print("=============")

#see player hands
for player in players :
  print("=============")
  player.displayHand()

DOMINO SET
%%%%%%
0
=
0
%%%%%%
0
=
1
%%%%%%
0
=
2
%%%%%%
0
=
3
%%%%%%
0
=
4
%%%%%%
0
=
5
%%%%%%
0
=
6
%%%%%%
1
=
1
%%%%%%
1
=
2
%%%%%%
1
=
3
%%%%%%
1
=
4
%%%%%%
1
=
5
%%%%%%
1
=
6
%%%%%%
2
=
2
%%%%%%
2
=
3
%%%%%%
2
=
4
%%%%%%
2
=
5
%%%%%%
2
=
6
%%%%%%
3
=
3
%%%%%%
3
=
4
%%%%%%
3
=
5
%%%%%%
3
=
6
%%%%%%
4
=
4
%%%%%%
4
=
5
%%%%%%
4
=
6
%%%%%%
5
=
5
%%%%%%
5
=
6
%%%%%%
6
=
6
DOMINO SET SIZE
28
POST DEAL DOMINO SET
POST DEAL DOMINO SET SIZE
0
Seby  has  7  dominos left.
%%%%%%
2
=
5
%%%%%%
1
=
4
%%%%%%
1
=
1
%%%%%%
0
=
0
%%%%%%
5
=
5
%%%%%%
1
=
5
%%%%%%
1
=
6
Mino  has  7  dominos left.
%%%%%%
2
=
4
%%%%%%
2
=
2
%%%%%%
0
=
1
%%%%%%
5
=
6
%%%%%%
0
=
3
%%%%%%
3
=
4
%%%%%%
3
=
3
Tita  has  7  dominos left.
%%%%%%
2
=
6
%%%%%%
3
=
5
%%%%%%
6
=
6
%%%%%%
4
=
5
%%%%%%
4
=
4
%%%%%%
0
=
2
%%%%%%
3
=
6
Gordo  has  7  dominos left.
%%%%%%
0
=
5
%%%%%%
1
=
3
%%%%%%
1
=
2
%%%%%%
4
=
6
%%%%%%
0
=
4
%%%%%%
2
=
3
%%%%%%
0
=
6


In [58]:
def reorder_players(index, plys):
    if index >= len(plys):
        return plys  # Index out of range, return the original list
    # Split the list into two parts
    first_part = plys[:index]
    second_part = plys[index:]
    # Concatenate the second part with the first part
    reordered_lst = second_part + first_part
    return reordered_lst

players = reorder_players(3, players)

i = 0
for output in cycle(players):
    i += 1
    print(output.name)
    if i >= 10 :
      break


Seby
Mino
Tita
Gordo
Seby
Mino
Tita
Gordo
Seby
Mino
